# Problem 26
A unit fraction contains 1 in the numerator. The decimal representation of the unit fractions with denominators 2 to 10 are given:

1/2	= 	0.5
1/3	= 	0.(3)
1/4	= 	0.25
1/5	= 	0.2
1/6	= 	0.1(6)
1/7	= 	0.(142857)
1/8	= 	0.125
1/9	= 	0.(1)
1/10	= 	0.1
Where 0.1(6) means 0.166666..., and has a 1-digit recurring cycle. It can be seen that 1/7 has a 6-digit recurring cycle.

Find the value of d < 1000 for which 1/d contains the longest recurring cycle in its decimal fraction part.

---

A usefull page:
https://en.wikipedia.org/wiki/Repeating_decimal

In [29]:
for i in range(1, 11):
    print('i', i, '1/i', 1/i)

i 1 1/i 1.0
i 2 1/i 0.5
i 3 1/i 0.3333333333333333
i 4 1/i 0.25
i 5 1/i 0.2
i 6 1/i 0.16666666666666666
i 7 1/i 0.14285714285714285
i 8 1/i 0.125
i 9 1/i 0.1111111111111111
i 10 1/i 0.1


We see that 3, 6, 7 and 9 have a recurrance

In [47]:
def is_prime(num):
    if num > 1:
        # check for factors
        for i in range(2,int(num**(.5))+1):
            if (num % i) == 0:
                return False
        else:
            return True

In [60]:
primes = list(filter(lambda x: is_prime(x), range(1000)))

In [75]:
1/3, 1/7

(0.3333333333333333, 0.14285714285714285)

In [87]:
1/1.21

0.8264462809917356

In [71]:
for prime in primes:
    print(f'{prime}, {1/int(prime):.13}')

2, 0.5
3, 0.3333333333333
5, 0.2
7, 0.1428571428571
11, 0.09090909090909
13, 0.07692307692308
17, 0.05882352941176
19, 0.05263157894737
23, 0.04347826086957
29, 0.03448275862069
31, 0.03225806451613
37, 0.02702702702703
41, 0.02439024390244
43, 0.02325581395349
47, 0.02127659574468
53, 0.0188679245283
59, 0.01694915254237
61, 0.01639344262295
67, 0.01492537313433
71, 0.01408450704225
73, 0.01369863013699
79, 0.0126582278481
83, 0.01204819277108
89, 0.01123595505618
97, 0.01030927835052
101, 0.00990099009901
103, 0.009708737864078
107, 0.009345794392523
109, 0.009174311926606
113, 0.008849557522124
127, 0.007874015748031
131, 0.00763358778626
137, 0.007299270072993
139, 0.007194244604317
149, 0.006711409395973
151, 0.006622516556291
157, 0.006369426751592
163, 0.006134969325153
167, 0.005988023952096
173, 0.005780346820809
179, 0.005586592178771
181, 0.005524861878453
191, 0.005235602094241
193, 0.005181347150259
197, 0.00507614213198
199, 0.005025125628141
211, 0.004739336492891
223, 0